# 1. Introduction

The purpose of this notebook is to see when GPyTorch fails for a simple data generating processes when inverse gamma priors are used. Four data sets are generated, normalized, and saved to .csv for later comparison using GPML. A Gaussian process regression model with zero mean and isotropic squared exponential kernel are trained on the data using GPyTorch. Model parameters are learned using an HMC sampler.

In [1]:
pip install gpytorch pyro-ppl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 310 kB 8.0 MB/s 
     |████████████████████████████████| 718 kB 39.4 MB/s 
  Created wheel for gpytorch: filename=gpytorch-1.6.0-py2.py3-none-any.whl size=509889 sha256=9fd63d0b1f72ec971f223a4fd2b879db1497f2a7602bbb61cd67c8051d4c4e7a
  Stored in directory: /root/.cache/pip/wheels/66/b5/89/34c06ad393a6feb72b4cdde46d0f1c667f3e2632960f9df109
Successfully built gpytorch


In [2]:
import math
import torch
import gpytorch
from gpytorch.priors import GammaPrior
import pyro
import numpy as np
import pandas as pd
from pyro.infer.mcmc import NUTS, MCMC, HMC
from matplotlib import pyplot as plt
from matplotlib import cm
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files

pyro.set_rng_seed(12345)
rng = np.random.default_rng(12345)

%matplotlib inline
%load_ext autoreload
%autoreload 2

Mounted at /content/drive


# 2. Data Generating Process

I generate four data sets each with a sample size of $n=101$. The single covariate $x$ is generated i.i.d. from the standard normal distribution. Measurement error is also generated from the standard normal distribution and scaled for two of the four data generating processes.

Simple Linear DGP with low noise: 

$y = 1 - x + (0.1)*u$

Simple Linear DGP with high noise:

$y = 1 - x + u$

Simple Cubic DGP with low noise:

$y = 1 - x + x^2 - x^3 + (0.1)*u$

Simple Cubic DGP with high noise:

$y = 1 - x + x^2 - x^3 + u$

In [3]:
# Generate data for the simulations
n = 101
intercept = 1
slope = -1
noise_scale = 0.1

x = np.random.normal(size=(n,1))
u = np.random.normal(size=(n,1))

simple_linear_lownoise = intercept*np.ones((n,1)) + slope*x + noise_scale*u # y1
simple_linear_highnoise = intercept*np.ones((n,1)) + slope*x + u #y2
simple_cubic_lownoise = intercept*np.ones((n,1)) + slope*x + np.power(x, 2) + slope*np.power(x, 3) + noise_scale*u #y3
simple_cubic_highnoise = intercept*np.ones((n,1)) + slope*x + np.power(x, 2) + slope*np.power(x, 3) + u #y4

# Normalize the training inputs
train_x = (x - x.mean())/x.std()

# Normalize the training outputs
train_y1 = (simple_linear_lownoise - simple_linear_lownoise.mean())/simple_linear_lownoise.std()
train_y2 = (simple_linear_highnoise - simple_linear_highnoise.mean())/simple_linear_highnoise.std()
train_y3 = (simple_cubic_lownoise - simple_cubic_lownoise.mean())/simple_cubic_lownoise.std()
train_y4 = (simple_cubic_highnoise - simple_cubic_highnoise.mean())/simple_cubic_highnoise.std()


In [ ]:
# Download the data as separate .csv files for use in GPML

# train_y1 and train_x
df1 = pd.DataFrame(np.column_stack((train_y1, train_x)))
df1.rename(columns={0:'y1', 1:'x'}, inplace=True)
df1.to_csv('simple_linear_lownoise.csv', encoding = 'utf-8-sig') 
files.download('simple_linear_lownoise.csv')

# train_y2 and train_x
df2 = pd.DataFrame(np.column_stack((train_y2, train_x)))
df2.rename(columns={0:'y2', 1:'x'}, inplace=True)
df2.to_csv('simple_linear_highnoise.csv', encoding = 'utf-8-sig') 
files.download('simple_linear_highnoise.csv')

# train_y3 and train_x
df3 = pd.DataFrame(np.column_stack((train_y3, train_x)))
df3.rename(columns={0:'y3', 1:'x'}, inplace=True)
df3.to_csv('simple_cubic_lownoise.csv', encoding = 'utf-8-sig') 
files.download('simple_cubic_lownoise.csv')

# train_y4 and train_x
df4 = pd.DataFrame(np.column_stack((train_y4, train_x)))
df4.rename(columns={0:'y4', 1:'x'}, inplace=True)
df4.to_csv('simple_cubic_highnoise.csv', encoding = 'utf-8-sig') 
files.download('simple_cubic_highnoise.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
# Specify the model, likelihood, and parameters for prior distributions
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ZeroMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Specify the pyro_model
def pyro_model(x, y):
    with gpytorch.settings.fast_computations(False, False, False):
        sampled_model = model.pyro_sample_from_prior()
        output = sampled_model.likelihood(sampled_model(x))
        pyro.sample("obs", output, obs=y)
    return y

# Set up the sampler
NUM_SAMPLES = 200
NUM_WARMUP = 100
NUM_CHAINS = 1
kernel = NUTS(pyro_model)
mcmc_run = MCMC(kernel, 
                num_samples=NUM_SAMPLES, 
                warmup_steps=NUM_WARMUP, 
                num_chains = NUM_CHAINS,
                disable_progbar=False)

# 3. Simulation Experiment

Anything above this text is common to every model that attempts to run below. In the following frames, each cell attempts use Pyro with Exact GP inference for different parameter inverse gamma parameters. All models will take 200 samples with 100 burn-in over one chain.

Here is a list of the gamma parameter values that I test.

1.  GammaPrior(1, 1/5)
2.  GammaPrior(1, 1/2)
3.  GammaPrior(1, 1)
4.  GammaPrior(1, 3/2)
5.  GammaPrior(1, 2)
6.  GammaPrior(2, 1)
7.  GammaPrior(2, 3/2)
8.  GammaPrior(2,2)
9.  GammaPrior(3, 1)
10. GammaPrior(3, 1/2)


In [5]:
# Before starting, convert the numpy arrays to torch tensors

train_x = torch.from_numpy(train_x)
train_y1 = torch.from_numpy(train_y1)
train_y2 = torch.from_numpy(train_y2)
train_y3 = torch.from_numpy(train_y3)
train_y4 = torch.from_numpy(train_y4)


In [ ]:
# Dropping the unused tensor dimension won't work either.

#train_x = train_x.reshape((-1,))
#train_y1 = train_y1.reshape((-1,))
#train_y2 = train_y2.reshape((-1,))
#train_y3 = train_y3.reshape((-1,))
#train_y4 = train_y4.reshape((-1,))

### 1.  GammaPrior(1, 1/5)

In [6]:
# y1 --- Simple linear DGP with low noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y1, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(1, 1/5), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(1, 1/5), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(1, 1/5), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y1)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/lazy/triangular_lazy_tensor.py:130: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:1672.)
  res = torch.triangular_solve(right_tensor, self.evaluate(), upper=self.upper).solution
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
Sample: 100%|██████████| 300/300 [24:31,  4.90s/it, step size=1.03e-06, acc. prob=0.350]


In [7]:
# y2 --- Simple linear DGP with high noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y2, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(1, 1/5), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(1, 1/5), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(1, 1/5), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y2)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,


NotPSDError: ignored

In [8]:
# y3 --- Simple cubic DGP with low noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y3, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(1, 1/5), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(1, 1/5), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(1, 1/5), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y3)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,


NotPSDError: ignored

In [9]:
# y4 --- Simple cubic DGP with high noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y4, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(1, 1/5), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(1, 1/5), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(1, 1/5), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y4)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,


NotPSDError: ignored

### 2.  GammaPrior(1, 1/2)

In [10]:
# y1 --- Simple linear DGP with low noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y1, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(1, 1/2), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(1, 1/2), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(1, 1/2), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y1)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,


NotPSDError: ignored

In [11]:
# y2 --- Simple linear DGP with high noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y2, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(1, 1/2), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(1, 1/2), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(1, 1/2), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y2)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,


NotPSDError: ignored

In [12]:
# y3 --- Simple cubic DGP with low noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y3, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(1, 1/2), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(1, 1/2), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(1, 1/2), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y3)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,


NotPSDError: ignored

In [13]:
# y4 --- Simple cubic DGP with high noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y4, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(1, 1/2), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(1, 1/2), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(1, 1/2), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y4)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,


NotPSDError: ignored

### 3.  GammaPrior(1, 1)


In [14]:
# y1 --- Simple linear DGP with low noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y1, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(1, 1), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(1, 1), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(1, 1), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y1)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,


NotPSDError: ignored

In [15]:
# y2 --- Simple linear DGP with high noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y2, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(1, 1), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(1, 1), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(1, 1), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y2)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,


NotPSDError: ignored

In [16]:
# y3 --- Simple cubic DGP with low noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y3, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(1, 1), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(1, 1), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(1, 1), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y3)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,


NotPSDError: ignored

In [17]:
# y4 --- Simple cubic DGP with high noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y4, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(1, 1), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(1, 1), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(1, 1), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y4)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,


NotPSDError: ignored

### 4.  GammaPrior(1, 3/2)

In [18]:
# y1 --- Simple linear DGP with low noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y1, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(1, 3/2), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(1, 3/2), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(1, 3/2), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y1)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,


NotPSDError: ignored

In [19]:
# y2 --- Simple linear DGP with high noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y2, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(1, 3/2), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(1, 3/2), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(1, 3/2), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y2)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,


NotPSDError: ignored

In [20]:
# y3 --- Simple cubic DGP with low noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y3, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(1, 3/2), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(1, 3/2), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(1, 3/2), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y3)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,


NotPSDError: ignored

In [21]:
# y4 --- Simple cubic DGP with high noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y4, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(1, 3/2), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(1, 3/2), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(1, 3/2), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y4)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,


NotPSDError: ignored

### 5.  GammaPrior(1, 2)


In [22]:
# y1 --- Simple linear DGP with low noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y1, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(1, 2), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(1, 2), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(1, 2), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y1)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,


NotPSDError: ignored

In [23]:
# y2 --- Simple linear DGP with high noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y2, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(1, 2), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(1, 2), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(1, 2), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y2)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,


NotPSDError: ignored

In [ ]:
# y3 --- Simple cubic DGP with low noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y3, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(1, 2), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(1, 2), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(1, 2), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y3)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,


NanError: ignored

In [ ]:
# y4 --- Simple cubic DGP with high noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y4, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(1, 2), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(1, 2), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(1, 2), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y4)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,


NanError: ignored

### 6.  GammaPrior(2, 1)

In [ ]:
# y1 --- Simple linear DGP with low noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y1, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(2, 1), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(2, 1), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(2, 1), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y1)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,


NanError: ignored

In [ ]:
# y2 --- Simple linear DGP with high noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y2, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(2, 1), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(2, 1), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(2, 1), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y2)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,


NanError: ignored

In [ ]:
# y3 --- Simple cubic DGP with low noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y3, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(2, 1), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(2, 1), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(2, 1), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y3)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,


NanError: ignored

In [ ]:
# y4 --- Simple cubic DGP with high noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y4, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(2, 1), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(2, 1), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(2, 1), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y4)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,


NanError: ignored

### 7.  GammaPrior(2, 3/2)


In [24]:
# y1 --- Simple linear DGP with low noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y1, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(2, 3/2), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(2, 3/2), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(2, 3/2), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y1)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,


NotPSDError: ignored

In [25]:
# y2 --- Simple linear DGP with high noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y2, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(2, 3/2), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(2, 3/2), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(2, 3/2), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y2)

Warmup:   0%|          | 0/300 [01:00, ?it/s]
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,


NotPSDError: ignored

In [26]:
# y3 --- Simple cubic DGP with low noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y3, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(2, 3/2), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(2, 3/2), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(2, 3/2), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y3)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,


NotPSDError: ignored

In [27]:
# y4 --- Simple linear DGP with low noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y4, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(2, 3/2), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(2, 3/2), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(2, 3/2), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y4)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,


NotPSDError: ignored

### 8.  GammaPrior(2,2)

In [28]:
# y1 --- Simple linear DGP with low noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y1, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(2, 2), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(2, 2), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(2, 2), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y1)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,


NotPSDError: ignored

In [29]:
# y2 --- Simple linear DGP with high noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y2, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(2, 2), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(2, 2), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(2, 2), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y2)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,


NotPSDError: ignored

In [30]:
# y3 --- Simple cubic DGP with low noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y3, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(2, 2), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(2, 2), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(2, 2), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y3)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,


NotPSDError: ignored

In [31]:
# y4 --- Simple cubic DGP with high noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y4, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(2, 2), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(2, 2), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(2, 2), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y4)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,


NotPSDError: ignored

### 9.  GammaPrior(3, 1)

In [32]:
# y1 --- Simple linear DGP with low noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y1, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(3, 1), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(3, 1), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(3, 1), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y1)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,


NotPSDError: ignored

In [33]:
# y2 --- Simple linear DGP with high noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y2, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(2, 2), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(2, 2), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(2, 2), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y2)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,


NotPSDError: ignored

In [34]:
# y3 --- Simple cubic DGP with low noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y3, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(2, 2), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(2, 2), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(2, 2), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y3)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,


NotPSDError: ignored

In [35]:
# y4 --- Simple cubic DGP with high noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y4, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(2, 2), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(2, 2), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(2, 2), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y4)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,


NotPSDError: ignored

### 10. GammaPrior(3, 1/2)


In [36]:
# y1 --- Simple linear DGP with low noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y1, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(3, 1/2), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(3, 1/2), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(3, 1/2), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y1)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,


NotPSDError: ignored

In [37]:
# y2 --- Simple linear DGP with low noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y2, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(3, 1/2), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(3, 1/2), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(3, 1/2), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y2)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,


NotPSDError: ignored

In [38]:
# y3 --- Simple cubic DGP with low noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y3, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(3, 1/2), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(3, 1/2), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(3, 1/2), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y3)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,


NotPSDError: ignored

In [39]:
# y4 --- Simple cubic DGP with high noise

likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
model = ExactGPModel(train_x, train_y4, likelihood)

model.covar_module.base_kernel.register_prior("lengthscale_prior", GammaPrior(3, 1/2), "lengthscale")
model.covar_module.register_prior("outputscale_prior", GammaPrior(3, 1/2), "outputscale")
likelihood.register_prior("noise_prior", GammaPrior(3, 1/2), "noise")

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# See the error below this, if the error exists. No error means it finished training.
mcmc_run.run(train_x, train_y4)

Warmup:   0%|          | 0/300 [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  NumericalWarning,
/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,


NotPSDError: ignored